현재 상태:

Tumor 디렉토리:

    - SRR30510137: ✅ 완료

    - SRR30510151: ✅ 완료

    - SRR30510181: ✅ 완료

Normal 디렉토리:

    - SRR28602760: ✅ 완료

    - SRR28602761: ✅ 완료

    - SRR28602767: ✅ 완료

각 디렉토리 별로 아래 단계가 완료된 상태입니다.

- fastqc (품질 체크)

- trim_galore (트리밍)

- STAR (정렬 및 BAM 파일 생성)

- featureCounts (유전자 발현량 정량화, counts.txt 생성)

In [9]:
import pandas as pd
import glob
import os

tumor_paths = glob.glob('./Tumor/*/counts.txt')
normal_paths = glob.glob('./Normal/*/counts.txt')

usecols = ['Geneid', 'SRR']

counts_tumor = []
for path in tumor_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_tumor.append(df.set_index('Geneid'))

counts_tumor_df = pd.concat(counts_tumor, axis=1)

counts_normal = []
for path in normal_paths:
    sample_name = os.path.basename(os.path.dirname(path))
    df = pd.read_csv(path, sep='\t', comment='#', usecols=[0, 6], names=['Geneid', sample_name], header=0)
    counts_normal.append(df.set_index('Geneid'))

counts_normal_df = pd.concat(counts_normal, axis=1)

combined_counts_df = pd.concat([counts_tumor_df, counts_normal_df], axis=1)
combined_counts_df.columns.name = 'Sample'

print(combined_counts_df.head())

Sample           SRR30510137  SRR30510151  SRR30510181  SRR28602760  \
Geneid                                                                
ENSG00000279928            6            0            2            2   
ENSG00000228037            0            0            7           15   
ENSG00000142611         2403          735          168         1128   
ENSG00000284616            0            0            0            0   
ENSG00000157911         1369          537          889          805   

Sample           SRR28602761  SRR28602767  
Geneid                                     
ENSG00000279928            2            6  
ENSG00000228037            8            6  
ENSG00000142611         1247          504  
ENSG00000284616            0            0  
ENSG00000157911         1177         1301  


In [10]:
combined_counts_df.columns = [
    'Tumor_SRR30510137',
    'Tumor_SRR30510151',
    'Tumor_SRR30510181',
    'Normal_SRR28602760',
    'Normal_SRR28602761',
    'Normal_SRR28602767'
]

In [11]:
combined_counts_df.head()

,Tumor_SRR30510137,Tumor_SRR30510151,Tumor_SRR30510181,Normal_SRR28602760,Normal_SRR28602761,Normal_SRR28602767
Geneid,,,,,,
ENSG00000279928,6,0,2,2,2,6
ENSG00000228037,0,0,7,15,8,6
ENSG00000142611,2403,735,168,1128,1247,504
ENSG00000284616,0,0,0,0,0,0
ENSG00000157911,1369,537,889,805,1177,1301


In [6]:
!pip uninstall gprofiler
!pip install gprofiler-official

Found existing installation: gprofiler 1.2.2
Uninstalling gprofiler-1.2.2:
  Would remove:
    /home/min/anaconda3/envs/BIO/lib/python3.10/site-packages/gprofiler-1.2.2.dist-info/*
    /home/min/anaconda3/envs/BIO/lib/python3.10/site-packages/gprofiler/*
Proceed (Y/n)? ERROR: Operation cancelled by user
^C
  Using cached gprofiler_official-1.0.0-py3-none-any.whl.metadata (11 kB)
Using cached gprofiler_official-1.0.0-py3-none-any.whl (9.3 kB)


### Differential Expression Analysis (발현차이 분석)

- DESeq2, edgeR, limma-voom이 대표적입니다.

- 그룹간(종양 vs 정상) 유의하게 발현이 다른 유전자를 찾음

- Fold-change 및 adjusted p-value가 중요합니다.

In [4]:
# 이미 설치된 상태에서 실행 가능한 코드
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

r = robjects.r
r('''
library(DESeq2)
run_DESeq2 <- function(countData, condition) {
  colData <- data.frame(row.names=colnames(countData), condition=condition)
  dds <- DESeqDataSetFromMatrix(countData = countData, colData = colData, design = ~ condition)
  dds <- DESeq(dds)
  res <- results(dds)
  return(as.data.frame(res))
}
''')

run_DESeq2 = r['run_DESeq2']

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


R[write to console]: 
Attaching package: ‘S4Vectors’


R[write to console]: The following object is masked from ‘package:utils’:

    findMatches


R[write to console]: The following 

In [7]:
from gprofiler import GProfiler
gp = GProfiler(return_dataframe=True)
results = gp.profile(organism='hsapiens', query=['BRCA1', 'TP53', 'EGFR'])


ImportError: cannot import name 'GProfiler' from 'gprofiler' (/home/min/anaconda3/envs/BIO/lib/python3.10/site-packages/gprofiler/__init__.py)